# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
df_numbered = df.copy()
label_encoder = LabelEncoder()
label_encoder.fit(df_numbered["loan_status"])


LabelEncoder()

In [5]:
df_numbered["loan_status_#"] = label_encoder.transform(df_numbered["loan_status"])

In [6]:
label_encoder.fit(df_numbered["homeowner"])

LabelEncoder()

In [8]:
df_numbered["homeowner_#"] = label_encoder.transform(df_numbered["homeowner"])

In [9]:
df_numbered.drop(columns = ["homeowner", "loan_status"], inplace = True)
df_numbered.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status_#,homeowner_#
0,10700.0,7.672,52800,0.431818,5,1,22800,1,1
1,8400.0,6.692,43600,0.311927,3,0,13600,1,1
2,9000.0,6.963,46100,0.349241,3,0,16100,1,2
3,10700.0,7.664,52700,0.430740,5,1,22700,1,1
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0


# Split the Data into Training and Testing

In [10]:
# Create our features
X = df_numbered.copy()
X.drop("loan_status_#", axis=1, inplace=True)
X.head()

# Create our target
y = df_numbered["loan_status_#"]

In [11]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_#
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.606144
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.667811
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,1.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,2.000000


In [12]:
# Check the balance of our target values
y.value_counts()

1    75036
0     2500
Name: loan_status_#, dtype: int64

In [13]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()


In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [16]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9481182566723452

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  541,    59],
       [  102, 18682]])

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.90      0.99      0.87      0.95      0.89       600
          1       1.00      0.99      0.90      1.00      0.95      0.91     18784

avg / total       0.99      0.99      0.90      0.99      0.95      0.90     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [21]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE

Counter(y_resampled)

Counter({1: 56252, 0: 56252})

In [22]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_2 = LogisticRegression(solver='lbfgs', random_state=1)
model_2.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_2 = model_2.predict(X_test)
balanced_accuracy_score(y_test, y_pred_2)

0.9945453577512777

In [24]:
# Display the confusion matrix
# YOUR CODE HERE

confusion_matrix(y_test, y_pred_2)


array([[  597,     3],
       [  111, 18673]])

In [25]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_2))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.99      0.99      0.91      0.99      0.99       600
          1       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [26]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

X_resampled_2, y_resampled_2 = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)


# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_2)

Counter({1: 56252, 0: 56252})

In [27]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_3 = LogisticRegression(solver='lbfgs', random_state=1)
model_3.fit(X_resampled_2, y_resampled_2)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_3 = model_3.predict(X_test)
balanced_accuracy_score(y_test, y_pred_3)

0.9945453577512777

In [29]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_3)

array([[  597,     3],
       [  111, 18673]])

In [30]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_3))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.99      0.99      0.91      0.99      0.99       600
          1       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled_3, y_resampled_3 = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_3)

Counter({0: 1900, 1: 1900})

In [32]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_4 = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model_4.fit(X_resampled_3, y_resampled_3)

LogisticRegression(max_iter=2000, random_state=1)

In [33]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred_4 = model_4.predict(X_test)
balanced_accuracy_score(y_test, y_pred_4)

0.9812386428165816

In [34]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_4)

array([[  581,    19],
       [  110, 18674]])

In [35]:
# Print the imbalanced classification report
# YOUR CODE HERE

print(classification_report_imbalanced(y_test, y_pred_4))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.97      0.99      0.90      0.98      0.96       600
          1       1.00      0.99      0.97      1.00      0.98      0.97     18784

avg / total       0.99      0.99      0.97      0.99      0.98      0.96     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [37]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled_4, y_resampled_4 = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled_4)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_4)

Counter({0: 55563, 1: 55898})

In [38]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_5 = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model_5.fit(X_resampled_4, y_resampled_4)

LogisticRegression(max_iter=2000, random_state=1)

In [39]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred_5 = model_5.predict(X_test)
balanced_accuracy_score(y_test, y_pred_5)

0.9952988358886996

In [40]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_5)

array([[  598,     2],
       [  114, 18670]])

In [41]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_5))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      1.00      0.99      0.91      1.00      0.99       600
          1       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

## 1. Which model had the best balanced accuracy score?

   The SMOTEENN has the best balanced accuracy score of 0.9952988358886996.

## 2. Which model had the best recall score?

    The SMOTEENN has the best recall score of .99.

## 3. Which model had the best geometric mean score?

    The SMOTEENN has the best geometric mean score of 1.00.
